# Adoption DFJ

In [1]:
import pandas as pd  # for the data structures to store and manipulate tables
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools
import boto3  # for working with AWS

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

database = "familyman_dev_v2"
snapshot_date = "2021-08-19"
database_derived = "familyman_derived_dev_v1"

fcsq_database = "fcsq"

s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

## Adoption_Disposals_Orders table

### Create the Adoption_Disposals_Orders temp table 

In [2]:
create_Adoption_Disposals_Orders = f"""
SELECT YEAR, QUARTER, Court, count(*) as Disposals
FROM fcsq.adopt_disposals5_adoption
WHERE Year>2010
GROUP BY Year,Quarter,Court
ORDER BY Year,Quarter,Court;
"""

pydb.create_temp_table(create_Adoption_Disposals_Orders,'Adoption_Disposals_Orders');



#### Adoption_Disposals_Orders validation

In [3]:
#Adoption_Disposals_Orders_count = pydb.read_sql_query("select count(*) as count from __temp__.Adoption_Disposals_Orders")
#Adoption_Disposals_Orders_count

## adopt_disposal_case_count_A table

### Drop the adopt_disposal_case_count_A table if it already exists and remove its data from the S3 bucket

In [4]:
drop_adopt_disposal_case_count_A = f"""
DROP TABLE IF EXISTS fcsq.adopt_disposal_case_count_A;
"""
pydb.start_query_execution_and_wait(drop_adopt_disposal_case_count_A)

# clean up previous adopt_disposal_case_count_A files
bucket.objects.filter(Prefix="fcsq_processing/Adoption/adopt_disposal_case_count_A/").delete();

### Create the adopt_disposal_case_count_A table in Athena

In [5]:
create_adopt_disposal_case_count_A = f"""
SELECT 
t1.CASE_NUMBER, 
Min(t1.Receipt_date) AS MinOfReceipt_date,
t1.Court 
FROM fcsq.adopt_disposals5_adoption as t1
GROUP BY t1.CASE_NUMBER, t1.Court;

"""

pydb.create_temp_table(create_adopt_disposal_case_count_A,'adopt_disposal_case_count_A');



#### adopt_disposal_case_count_A validation

In [6]:
#adopt_disposal_case_count_A_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_disposal_case_count_A")
#adopt_disposal_case_count_A_count

### Create the adopt_disposal_case_count_B table in Athena

In [7]:
create_adopt_disposal_case_count_B = f"""
SELECT 
t1.CASE_NUMBER, 
Year (t1.MinOfReceipt_date) AS YEAR,
CASE WHEN Month(t1.MinOfReceipt_date)<4
    THEN 1
        WHEN Month(t1.MinOfReceipt_date)<7
        THEN 2
            WHEN Month(t1.MinOfReceipt_date)<10
            THEN 3

ELSE 4
END AS Quarter,
t1.MinOfReceipt_date,
t1.Court 
FROM __temp__.adopt_disposal_case_count_A  as t1;

"""

pydb.create_temp_table(create_adopt_disposal_case_count_B,'adopt_disposal_case_count_B')



#### adopt_disposal_case_count_B validation

In [8]:
#adopt_disposal_case_count_B_count = pydb.read_sql_query("select count(*) as count from __temp__.adopt_disposal_case_count_B")
#adopt_disposal_case_count_B_count

## Adoption_Disposals_Case table

### Create the Adoption_Disposals_Case table in Athena

In [9]:
create_Adoption_Disposals_Case = f"""
SELECT Year,Quarter,Court,count(*) as Case_End
FROM __temp__.adopt_disposal_case_count_B
WHERE year>2010
GROUP BY Year,Quarter,Court
ORDER BY Year,Quarter,Court;
"""

pydb.create_temp_table(create_Adoption_Disposals_Case,'Adoption_Disposals_Case');



#### Adoption_Disposals_Case validation

In [10]:
#Adoption_Disposals_Case_count = pydb.read_sql_query("select count(*) as count from __temp__.Adoption_Disposals_Case")
#Adoption_Disposals_Case_count

## adoption_disposals_merge table

### Create the adoption_disposals_merge table in Athena

In [11]:
create_adoption_disposals_merge = f"""
SELECT
t1.year,
t1.quarter,
t1.court,
t1.Disposals,
t2.Case_End,
'Adoption' as Category,
'End' as Stage
FROM __temp__.adoption_disposals_orders t1 
INNER JOIN 
__temp__.adoption_disposals_case t2
ON t1.year = t2.year AND t1.quarter = t2.quarter AND t1.court = t2.court
where t1.year <> 2021 or t1.quarter <> 3;
"""

pydb.create_temp_table(create_adoption_disposals_merge,'adoption_disposals_merge');



#### adoption_disposals_merge validation

In [12]:
#adoption_disposals_merge_count = pydb.read_sql_query("select * from __temp__.adoption_disposals_merge ORDER BY Year,Quarter,Court;")
#adoption_disposals_merge_count

## adoption_disposals_format  table

### Create the adoption_disposals_format  table in Athena

In [13]:
create_adoption_disposals_format  = f"""
SELECT
Category,
Stage,
Year,
Quarter,
Court,
Disposals as Count,
Case_end as Cases
FROM
__temp__.adoption_disposals_merge;
"""

pydb.create_temp_table(create_adoption_disposals_format,'adoption_disposals_format');



#### adoption_disposals_format  validation

In [14]:
#adoption_disposals_format_count = pydb.read_sql_query("select count(*) as count from __temp__.adoption_disposals_format ")
#adoption_disposals_format_count

## Adoption_Applications_Ordera table

### Create the Adoption_Applications_Ordera table in Athena

In [15]:
create_Adoption_Applications_Ordera = f"""
SELECT YEAR, QUARTER, Court, count(*) as Apps
FROM fcsq.adopt_apps_6_adoptions_only
WHERE Year>2010
GROUP BY Year,Quarter,Court
ORDER BY Year,Quarter,Court;
"""

pydb.create_temp_table(create_Adoption_Applications_Ordera,'Adoption_Applications_Ordera');



#### Adoption_Applications_Ordera validation

In [16]:
#Adoption_Applications_Ordera_count = pydb.read_sql_query("select count(*) as count from __temp__.Adoption_Applications_Ordera")
#Adoption_Applications_Ordera_count

## Application_case_count_C table

### Create the Application_case_count_C table in Athena

In [17]:
create_Application_case_count_C = f"""
SELECT 
t1.CASE_NUMBER, 
Min(t1.App_date) AS MinOfApp_date,
t1.Court 
FROM fcsq.adopt_apps_6_adoptions_only as t1
GROUP BY t1.CASE_NUMBER, t1.Court;

"""

pydb.create_temp_table(create_Application_case_count_C,'Application_case_count_C');



#### Application_case_count_C validation

In [18]:
#Application_case_count_C_count = pydb.read_sql_query("select count(*) as count from __temp__.Application_case_count_C")
#Application_case_count_C_count

### Create the Application_case_count_D table in Athena

In [19]:
create_Application_case_count_D = f"""
SELECT 
t1.CASE_NUMBER, 
Year (t1.MinOfApp_date) AS YEAR,
CASE WHEN Month(t1.MinOfApp_date)<4
    THEN 1
        WHEN Month(t1.MinOfApp_date)<7
        THEN 2
            WHEN Month(t1.MinOfApp_date)<10
            THEN 3

ELSE 4
END AS Quarter,
t1.MinOfApp_date,
t1.Court 
FROM __temp__.application_case_count_C  as t1;

"""

pydb.create_temp_table(create_Application_case_count_D,'Application_case_count_D');



#### Application_case_count_D validation

In [20]:
#Application_case_count_D_count = pydb.read_sql_query("select count(*) as count from __temp__.Application_case_count_D")
#Application_case_count_D_count

## Adoption_Applications_Case table

### Create the Adoption_Applications_Case table in Athena

In [21]:
create_Adoption_Applications_Case = f"""
SELECT Year,Quarter,Court,count(*) as Case_Count
FROM __temp__.Application_case_count_D
WHERE year>2010
GROUP BY Year,Quarter,Court
ORDER BY Year,Quarter,Court;
"""

pydb.create_temp_table(create_Adoption_Applications_Case,'Adoption_Applications_Case');



#### Adoption_Applications_Case validation

In [22]:
#Adoption_Applications_Case_count = pydb.read_sql_query("select count(*) as count from __temp__.Adoption_Applications_Case")
#Adoption_Applications_Case_count

## adoption_Applications_merge table

### Create the adoption_Applications_merge table in Athena

In [23]:
create_adoption_Applications_merge = f"""
SELECT
t1.year,
t1.quarter,
t1.court,
t1.apps,
t2.Case_Count,
'Adoption' as Category,
'Start' as Stage
FROM __temp__.Adoption_Applications_Ordera t1 
INNER JOIN 
__temp__.Adoption_Applications_Case t2
ON t1.year = t2.year AND t1.quarter = t2.quarter AND t1.court = t2.court
where t1.year <> 2021 or t1.quarter <> 3;
"""

pydb.create_temp_table(create_adoption_Applications_merge,'adoption_Applications_merge');



#### adoption_Applications_merge validation

In [24]:
#adoption_Applications_merge_count = pydb.read_sql_query("select * from __temp__.adoption_Applications_merge ORDER BY Year,Quarter,Court;")
#adoption_Applications_merge_count

## adoption_Applications_format  table

### Create the adoption_Applications_format  table in Athena

In [25]:
create_adoption_Applications_format  = f"""
SELECT
Category,
Stage,
Year,
Quarter,
Court,
apps as Count,
Case_Count as Cases
FROM
__temp__.adoption_Applications_merge;
"""

pydb.create_temp_table(create_adoption_Applications_format,'adoption_Applications_format');



#### adoption_Applications_format  validation

In [26]:
#adoption_Applications_format_count = pydb.read_sql_query("select * from __temp__.adoption_Applications_format ORDER BY Year,Quarter,Court")
#adoption_Applications_format_count

## adopt_court_level_append table

### Create the adopt_court_level_append table in Athena

In [27]:
create_adopt_court_level_append = f"""
SELECT * FROM __temp__.adoption_Applications_format 
UNION 
SELECT * FROM __temp__.adoption_Disposals_format 
ORDER BY Year,Quarter,Court
"""

pydb.create_temp_table(create_adopt_court_level_append,'adopt_court_level_append');



#### adopt_court_level_append validation

In [28]:
#adopt_court_level_append_count = pydb.read_sql_query("select * from __temp__.adopt_court_level_append")
#adopt_court_level_append_count

## court_lookup table

### Create the court_lookup table in Athena

In [29]:
create_court_lookup = f"""
SELECT 
code,
Region,
Region_Pre2014,
DFJ_New
FROM fcsq.court_mv_feb21_dfj;
"""

pydb.create_temp_table(create_court_lookup,'court_lookup');



#### court_lookup validation

In [30]:
#court_lookup_count = pydb.read_sql_query("select * from __temp__.court_lookup")
#court_lookup_count

## court_level_merge table

### Create the court_level_merge table in Athena

In [31]:
create_court_level_merge = f"""
SELECT
t1.Category,
t1.Stage,
t1.Year,
t1.Quarter,
t1.Count,
t1.Cases,
t2.DFJ_New,
Case when Year < 2014 then t2.Region_Pre2014
Else t2.Region
End As Final_Region
FROM 
__temp__.adopt_court_level_append t1
INNER JOIN
__temp__.court_lookup t2
ON CAST(t1.court as integer) = t2.code
where CAST(t1.court as integer) in (SELECT code from __temp__.court_lookup);
"""

pydb.create_temp_table(create_court_level_merge,'court_level_merge');



#### court_level_merge validation

In [32]:
#court_level_merge_count = pydb.read_sql_query("select * from __temp__.court_level_merge where year=2020 and quarter=3 and dfj_new='Carlisle DFJ' order by year,quarter,dfj_new")
#court_level_merge_count

## court_level_sum table

### Create the court_level_sum table in Athena

In [33]:
create_court_level_sum = f"""
SELECT
Category,
Stage,
Year,
Quarter,
DFJ_New,
Final_Region as region,
SUM(count) as count,
SUM(cases) as cases
FROM __temp__.court_level_merge
group by
Category,
Stage,
Year,
Quarter,
DFJ_New,
Final_Region;
"""

pydb.create_temp_table(create_court_level_sum,'court_level_sum');



#### court_level_sum validation

In [34]:
court_level_sum_count = pydb.read_sql_query("select * from __temp__.court_level_sum ORDER BY Category,Year,Quarter,region,DFJ_New,Stage")
court_level_sum_count.to_csv(path_or_buf = '~/FCSQ_data/adopt_dfj.csv')
#court_level_sum_count